### model: deeplab v3+ data: 遥感25分类
用generator生成数据

In [1]:
from model import Deeplabv3
model=Deeplabv3(weights=None,input_shape=(224,224,4),classes=23,backbone='xception')
model.load_weights('./models/deeplab_v3_s174_generate.hdf5')

Using TensorFlow backend.


In [2]:
from keras import optimizers
optimizer=optimizers.RMSprop(lr=1e-4)
#可以自定义损失函数和metrics
# loss_fn=softmax_sparse_crossentropy_ignoring_last_label
# metrics=[sparse_accuracy_ignoring_last_label]
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 4)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 112, 112, 32) 1152        input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 112, 112, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [3]:
import BatchDatsetReader as dataset
from keras.callbacks import ModelCheckpoint
import numpy as np
from tqdm import tqdm
validation_dataset_reader = dataset.BatchDatset(224, './data2/validation_image.npy', './data2/validation_annotation.npy')


# 将validation中的mask从(None,224,224)->(None,224,224,23)
for i in tqdm(range(15)):
    valid_images, valid_annotations = validation_dataset_reader.next_batch(100)
    if i==1:
        xs_val=np.concatenate((valid_images,temp_xs),axis=0)
        ys_val=np.concatenate((valid_annotations,temp_ys),axis=0)

    elif i>1:
        xs_val=np.concatenate((valid_images,xs_val),axis=0)
        ys_val=np.concatenate((valid_annotations,ys_val))        
    temp_xs,temp_ys=valid_images,valid_annotations

  0%|          | 0/15 [00:00<?, ?it/s]

Initializing Batch Dataset Reader...
(233, 224, 224, 4)
(233, 224, 224, 1)


100%|██████████| 15/15 [00:57<00:00,  3.81s/it]


In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='constant', cval=0)


def generateData(batch_size, data, mask):
    '''
    输入的是整个数据的images和masks，以及每次训练的batch_size数目，返回的是经过数据增强的batch_size批次大小的数组.
    其中mask的维度为(None,224,224),每次读取batch_size大小的images和masks时，做了(None,224,224)->(None,224,224,23)
    的转化，然后再做了数据增强。
    '''
    while True:
        train_data = []
        train_label = []
        batch = 0
        for i in (range(len(data))):

            batch += 1
            train_data.append(data[i])
            train_label.append(mask[i])
            if batch % batch_size == 0:
                train_data = np.array(train_data)
                train_label = np.array(train_label)
                ys_new_train = np.zeros(train_label.shape + (23,))
                for j in range(23):
                    ys_new_train[train_label == j, j] = 1
                train_label = ys_new_train.astype('int32')
                #                 yield (train_data,train_label)
                #datagen.flow()可以实现实时数据增强
#                 for x_batch, y_batch in datagen.flow(train_data, train_label, batch_size):
                    #                     print('i=%d'%i)
#                     yield (x_batch, y_batch)

                train_data = []
                train_label = []
                batch = 0
                break  #若是不停止，则generator会无限的对当前batch_size的图像生成经过数据增强的数据，
#                             这样就读不了下一个batch_size的图像


In [ ]:
from keras.callbacks import ModelCheckpoint


checkpoint=ModelCheckpoint('./models/deeplab_v3_s174_generate_5.hdf5',monitor='acc',save_best_only=True,verbose=1)
data=np.load('./data2/train_image.npy').swapaxes(1,2).swapaxes(2,3)
mask=np.load('./data2/train_annotation.npy')

val_data=np.load('./data2/validation_image.npy').swapaxes(1,2).swapaxes(2,3)
val_mask=np.load('./data2/validation_annotation.npy')

print(data.shape)
#每次训练是在一个batch_size上训练，生成器无限的返回batch_size大小的数据，直到运行到steps_per_epoch的时候会停止训练
history=model.fit_generator(generator=generateData(16,data,mask),steps_per_epoch=200,validation_data=(xs_val,ys_val),validation_steps=15,epochs=200,callbacks=[checkpoint])